<h4>Carga de Librerías</h4>

In [ ]:
import os
import arcpy
import pandas as pd
from arcgis.features import GeoAccessor, GeoSeriesAccessor

arcpy.env.overwriteOutput = True

<h4>Ubicación y registro de archivos .gdb dentro de múltiples directorios</h4>

In [ ]:
ruta_padre = r"D:\PUBLIC\PGAR\Insumos\Capas_Geograficas"

nombre_directorios = os.listdir(ruta_padre)

rutas_workspace = []

for directorio in nombre_directorios:
    ruta_directorio = os.path.join(ruta_padre, directorio)
    if os.path.isdir(ruta_directorio) == True:
        # Búsqueda de bases de datos dentro de segunda raíz de directorios
        nombre_subdirectorios = os.listdir(ruta_directorio)
        for subdirectorio in nombre_subdirectorios:
            if '.gdb' in subdirectorio:
                ruta_bd = os.path.join(ruta_directorio, subdirectorio)
                rutas_workspace.append(ruta_bd)

<h4>Parametrización de Código Temática</h4>

In [ ]:
nombre_campo_tematico = 'codigo_tematica'

# ? Parte del desarrollo, que por lo pronto, si o si se debe modificar manualmente
clasificacion_tematica = {'_1_POMCA': 1,
                          '_2_Cambio_Climatico': 2,
                          '_3_Cuencas_Hidrograficas': 3,
                          '_4_Suelos': 4,
                          '_5_Areas_Protegidas': 5,
                          '_6_Jurisdiccion_CAR': 6,
                          '_7_Rondas': 7,
                          '_8_Agua_Subterranea': 8,
                          '_9_Paramos': 9,
                          '_10_Analisis_Vulnerabilidad_Riesgo_Inundacion': 10,
                          '_11_Analisis_Vulnerabilidad_Riesgo_Incendio': 11,
                          '_12_Analisis_Vulnerabilidad_Riesgo_Avenida_Torrencial': 12,
                          '_13_Analisis_Vulnerabilidad_Riesgo_Remocion_Masa': 13,
                          '_14_Ecosistema':14,
                          '_15_Licencias_Ambientales':15,
                          '_16_Infraestuctura':16,
                          '_17_Desarrollo_Rural': 17,
                          '_18_Incendios':18,
                          '_19_Geologia_Geomorfologia':19,
                          '_20_Biodiversidad':20}

for espacio_trabajo in rutas_workspace:
    arcpy.env.workspace = espacio_trabajo
    for capa in arcpy.ListFeatureClasses():
        if 'nivel' not in capa:
            ruta_temporal = os.path.join(arcpy.env.workspace, capa)
            division_directorios = os.path.split(ruta_temporal)[0]
            carpetas = division_directorios.split("\\")[5]
            categorizacion_tema = carpetas.split("_")[1]
            # !!Crear Dominio!!, !!Revisar obligatoriedad!!
            arcpy.management.AddField(capa, nombre_campo_tematico, 'LONG', field_alias = 'Codigo Temática')
            for registro in clasificacion_tematica:
                if carpetas == registro:
                    arcpy.management.CalculateField(capa, nombre_campo_tematico, clasificacion_tematica[carpetas], 'PYTHON3')
                    print("Se actualiza de la capa {0}, la temática {1}, con el código: {2}".format(capa, carpetas, clasificacion_tematica[carpetas]))
        else:
            print("No se actualiza el código de temática para la capa {0}".format(capa))

<h4>Creación tabla -tbl_tematica-</h4>

In [ ]:
df_tematica = pd.DataFrame.from_dict(clasificacion_tematica, orient ='index')
df_tematica = df_tematica.reset_index()
df_tematica.rename({'index':'nombre_tematica', 0:'codigo_tematica'}, axis=1, inplace=True)

<h4>Exportar capa -tbl_tematica- a FileGDB</h4>

In [ ]:
ruta_capas_w_municipio = r"D:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\Local_BD_PGAR.gdb"
nombre_tabla_capa = 'tbl_tematica'
ruta_salida_tabla_tematica = os.path.join(ruta_capas_w_municipio, nombre_tabla_capa)
df_tematica.spatial.to_table(location = ruta_salida_tabla_tematica)
print("----------------------------- Se crea la tabla: -tbl_tematica- en Base de Datos -----------------------------")

<h4>Parametrización de Nombre Final Capa</h4>

In [ ]:
ruta_bd_local = r"D:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\Local_BD_PGAR.gdb"

nombre_campo_capa_final = 'nombre_capa_final'

for espacio_trabajo in rutas_workspace:
    arcpy.env.workspace = espacio_trabajo
    ruta_temporal = os.path.join(arcpy.env.workspace)
    division_directorios = os.path.split(ruta_temporal)[0]
    carpetas = division_directorios.split("\\")[5]
    categorizacion_tema = carpetas.split("_")[1]
    for capa in arcpy.ListFeatureClasses():
        if 'nivel' not in capa:
            arcpy.management.AddField(capa, nombre_campo_capa_final, 'TEXT', field_alias = 'Nombre Capa Final')
            capa_entrada = os.path.join(arcpy.env.workspace, capa)
            # Parametrizacion de temática en nombre de la capa
            ruta_archivos_a_local = os.path.join(ruta_bd_local, "_" + categorizacion_tema + "_" + capa)
            ruta_archivos_a_local = ruta_archivos_a_local.replace('__','_')
            nombre_capa_final = ruta_archivos_a_local.split('\\')[7]
            arcpy.management.CalculateField(capa, nombre_campo_capa_final, "'" + nombre_capa_final + "'", 'PYTHON3')
            print("Se crea y registra el nombre de la capa final sobre la capa: {0}".format(capa))

<h4>Borrado de capas en el archivo de consolidación local</h4>

In [ ]:
arcpy.env.workspace = ruta_bd_local
dataframe_interseccion = 'Interseccion_Municipio'

for capas in arcpy.ListFeatureClasses(feature_dataset=dataframe_interseccion):
    arcpy.Delete_management(capas)
    print("Se borra la capa {0}, alojada en el datafra {1}".format(capas, dataframe_interseccion))
    
for capas in arcpy.ListFeatureClasses():
    arcpy.Delete_management(capas)
    print("Se borra la capa {0} de la raiz de la BD".format(capas))    

<h4>Migración de capas a BD Local Temporal</h4>

In [ ]:
for espacio_trabajo in rutas_workspace:
    arcpy.env.workspace = espacio_trabajo
    ruta_temporal = os.path.join(arcpy.env.workspace)
    division_directorios = os.path.split(ruta_temporal)[0]
    carpetas = division_directorios.split("\\")[5]
    categorizacion_tema = carpetas.split("_")[1]
    for capa in arcpy.ListFeatureClasses():
        if 'nivel' not in capa:
            capa_entrada = os.path.join(arcpy.env.workspace, capa)
            # Parametrizacion de temática en nombre de la capa
            ruta_archivos_a_local = os.path.join(ruta_bd_local, "_" + categorizacion_tema + "_" + capa)
            ruta_archivos_a_local = ruta_archivos_a_local.replace('__','_')
            arcpy.management.CopyFeatures(capa_entrada, ruta_archivos_a_local)
            print("Se migra la capa: {0}".format(capa)) 

<h4>Paramtrización del código de departamento (DANE) por registro\capa</h4>

In [ ]:
arcpy.env.workspace = ruta_bd_local

ruta_interseccion_mpio = os.path.join(arcpy.env.workspace, dataframe_interseccion + '\\')

if arcpy.Exists(ruta_interseccion_mpio):
    for capas in arcpy.ListFeatureClasses(feature_dataset=dataframe_interseccion):
        arcpy.management.Delete(capas)
        
for capa in arcpy.ListFeatureClasses():
    capa_intersect_salida = capa + "_Intersect_Mpio"
    if capa != '_6_1_Jurisdiccion_CAR':
        arcpy.management.RepairGeometry(capa)
        arcpy.analysis.Intersect([capa,"_6_1_Jurisdiccion_CAR"], os.path.join(ruta_bd_local, dataframe_interseccion, capa_intersect_salida))
        print("Se interseca la capa {0}, con -Jurisdiccion_CAR-".format(capa))